# State
State是一個 字典（dict-like object），裡面存放流程運行需要的資料。
例如一個摘要 Agent 的 State：
```python
{
    "text": "原始文章內容…",
    "summary": "",
    "notes": []
}

```
每一步 Node（模型或工具）都會讀取與更新 State。

## State 的兩種寫法：TypedState vs DictState

### 方法 1：Typed State（推薦）
使用 TypedDict 或 pydantic 定義明確結構。

好處：
- 型別安全
- 方便 IDE 自動補全
- 結構清晰

In [ ]:
from typing import TypedDict, List

class ArticleState(TypedDict):
    text: str
    summary: str
    notes: List[str]

### 方法 2：Dict State
直接用 Python dict。

好處是寫起來快，但大型專案容易混亂。

In [ ]:
state = {
    "text": "",
    "summary": "",
    "notes": []
}

# Node
Node 是 LangGraph 中最重要的執行單元：

> 每個 Node 接收 state → 做某事 → 回傳要更新的 state 部分欄位。

在 LangGraph 裡：

- Node 不能直接修改整個 State
- Node 只能回傳「要更新的部分」
- Graph 會自動合併更新

In [ ]:
def summarize_node(state: ArticleState):
    summary = call_llm("請總結以下文章：" + state["text"])
    return {"summary": summary}

## Node 與 State 的互動流程
以 Node 更新 summary 為例：

### **流程**

1. Graph 執行到 summarize Node
2. 把 State 作為參數傳進去：`state`
3. Node 執行（呼叫模型或工具）
4. Node 回傳：
    
    ```python
    {"summary": "這是一段摘要"}
    
    ```
    
5. Graph 自動用回傳值更新 State

## Node 必須是純函式（pure function）：
- 不能修改原本的 State
- 不能有副作用（修改 global、寫檔案…除非必要）
- 只能讀 state → 回傳更新

## Node 更新規則
LangGraph 使用 Reducer 合併更新：

- State 是不可變（immutable）
- Node 回傳的部分 State 會 merge 進新的 State

In [ ]:
state = {"a": 1, "b": 2}

node 回傳 {"b": 3, "c": 4}

更新結果：
{"a":1, "b":3, "c":4}

## State 的更新方式
LangGraph 提供三種更新模式：

### Replace（預設）
Node 回傳值直接覆蓋同名欄位。

In [ ]:
from typing import TypedDict

class State(TypedDict):
    summary: str

In [ ]:
# 初始 State
state = {
    "summary": "舊的摘要"
}

In [ ]:
# Node 寫法
def update_summary(state: State):
    return {
        "summary": "新的摘要"
    }

In [ ]:
# 更新後 State
{
    "summary": "新的摘要"
}

### Append（列表合併）
- 使用 Annotated[list[T], operator.add]
- Node 回傳的 list → 自動 append 到原本 list

In [ ]:
from typing import TypedDict, Annotated
import operator

class State(TypedDict):
    logs: Annotated[list[str], operator.add]

In [ ]:
# 初始 State
state = {
    "logs": ["啟動流程"]
}

In [ ]:
# Node 寫法
def add_log(state: State):
    return {
        "logs": ["取得使用者輸入"]
    }

In [ ]:
# 更新後 State
{
    "logs": [
        "啟動流程",
        "取得使用者輸入"
    ]
}

### Deep Merge（巢狀結構局部更新）
- dict 內的 dict 可以「部分更新」
- 沒回傳的欄位會保留

In [ ]:
class State(TypedDict):
    user: dict

In [ ]:
# 初始 State
state = {
    "user": {
        "name": "Alice",
        "age": 30,
        "email": "alice@test.com"
    }
}

In [ ]:
# Node 只更新一個欄位
def update_age(state: State):
    return {
        "user": {
            "age": 31
        }
    }

In [ ]:
# 更新後 State
{
    "user": {
        "name": "Alice",
        "age": 31,
        "email": "alice@test.com"
    }
}
# 只改 age，其它欄位保留